In [1]:
import re
import os
import glob

In [2]:
class DependancyFinder:
    
    source_call_pattern = re.compile(r'source\s*\(\s*["\'](.*?)["\']\s*\)')
    
    def __init__(self, working_dir, file_path):
        self.working_dir = working_dir
        self.root_file = file_path
        self.dependencies = set()
        self.files_read = set()
        
    def __get_abs_path(self, file_path):
        if not os.path.isabs(file_path):
            return os.path.abspath(os.path.join(self.working_dir, file_path))
        return file_path
            
    
    def __find_deps(self, new_file):
        new_file = self.__get_abs_path(new_file)
        if new_file not in self.files_read:
            print(f"reading {new_file}")
            this_deps = set()
            with open(new_file, 'r') as fin:
                for line in fin:
                    matches = self.source_call_pattern.findall(line)
                    matches = [self.__get_abs_path(file) for file in matches]
                    this_deps.update(matches)
            self.dependencies.update(this_deps)
            self.files_read.add(new_file)
            return this_deps
                    
    def find(self):
        self.__find_deps(self.root_file)
        diff = self.dependencies.difference(self.files_read)
        itr = 0
        while len(diff) > 0 and itr < 10000:
            itr += 1
            for file in diff:
                self.__find_deps(file)
            diff = self.dependencies.difference(self.files_read)
        return self.dependencies

In [3]:
# Folder where the RProject file is using as the working directory (use the getwd() return value )
working_dir = '/Users/xzhao17/Documents/GitHub/xcj_hypothesis_test_cfd'

In [4]:
depfind = DependancyFinder(working_dir, './source_code/R/T_testing_hazel.R')
depfind.find()

reading /Users/xzhao17/Documents/GitHub/xcj_hypothesis_test_cfd/source_code/R/T_testing_hazel.R
reading /Users/xzhao17/Documents/GitHub/xcj_hypothesis_test_cfd/source_code/R/time_track_function.R
reading /Users/xzhao17/Documents/GitHub/xcj_hypothesis_test_cfd/source_code/R/T_testing_functions.R
reading /Users/xzhao17/Documents/GitHub/xcj_hypothesis_test_cfd/source_code/R/data_generator.R
reading /Users/xzhao17/Documents/GitHub/xcj_hypothesis_test_cfd/source_code/R/integral_penalty_function.R
reading /Users/xzhao17/Documents/GitHub/xcj_hypothesis_test_cfd/source_code/R/cfd_hypothesis_test.R
reading /Users/xzhao17/Documents/GitHub/xcj_hypothesis_test_cfd/source_code/R/run_gam_function.R


{'/Users/xzhao17/Documents/GitHub/xcj_hypothesis_test_cfd/source_code/R/T_testing_functions.R',
 '/Users/xzhao17/Documents/GitHub/xcj_hypothesis_test_cfd/source_code/R/cfd_hypothesis_test.R',
 '/Users/xzhao17/Documents/GitHub/xcj_hypothesis_test_cfd/source_code/R/data_generator.R',
 '/Users/xzhao17/Documents/GitHub/xcj_hypothesis_test_cfd/source_code/R/integral_penalty_function.R',
 '/Users/xzhao17/Documents/GitHub/xcj_hypothesis_test_cfd/source_code/R/run_gam_function.R',
 '/Users/xzhao17/Documents/GitHub/xcj_hypothesis_test_cfd/source_code/R/time_track_function.R'}

In [9]:
for file in depfind.dependencies:
    print(f"put {os.path.basename(file)}")
print("")

put time_track_function.R
put cfd_hypothesis_test.R
put run_gam_function.R
put integral_penalty_function.R
put T_testing_functions.R
put data_generator.R

